In [156]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import expit
from tqdm import tqdm

In [165]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test-public.csv")
test["Pair"] = list(zip(test.Source, test.Sink))

,Pair,Source,Sink,NCA,Exist,CN,AA,RA,JC,PA,KI,PR_s1,PR_s2
0,"(0, 356)",0,356,14,1,7,2.899858,0.628968,0.700000,72,0.079962,0.000220,0.000240
1,"(0, 1236)",0,1236,14,1,6,2.471649,0.531746,0.428571,96,0.075137,0.000220,0.000302
2,"(356, 1236)",356,1236,14,1,7,2.812086,0.587302,0.500000,108,0.074232,0.000240,0.000302
3,"(0, 1655)",0,1655,9,1,7,2.976054,0.668651,0.466667,112,0.083302,0.000220,0.000376
4,"(0, 1797)",0,1797,4,1,7,2.899858,0.628968,0.700000,72,0.081045,0.000220,0.000245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7093256,"(4089, 4091)",4089,4091,0,0,0,0.000000,0.000000,0.000000,35,0.000327,0.000223,0.000176
7093257,"(4089, 4092)",4089,4092,0,0,0,0.000000,0.000000,0.000000,25,0.000000,0.000223,0.000612
7093258,"(4090, 4091)",4090,4091,0,0,0,0.000000,0.000000,0.000000,42,0.006194,0.000201,0.000176
7093259,"(4090, 4092)",4090,4092,0,0,0,0.000000,0.000000,0.000000,30,0.000000,0.000201,0.000612


In [158]:
y = train.Exist
X = train[["NCA", "CN", "AA", "RA", "JC", "PA", "KI", "PR_s1", "PR_s2"]]

In [159]:
from sklearn.linear_model import LogisticRegression

# Code adapted from Worksheet04 to calculate the coefficients
clf = LogisticRegression(penalty='none')
clf.fit(X, y)
w_sklearn = np.r_[clf.intercept_, clf.coef_.squeeze()]
print("Weights according to scikit-learn: {}".format(w_sklearn))

Weights according to scikit-learn: [-7.47264209e+01  1.00675821e+02 -3.71960830e+00  1.86990933e+01
  8.54167241e+00  2.76017992e+00  3.55695393e-03  2.29468036e+00
 -6.06667680e-03 -4.68384149e-03]


In [160]:
X.insert(0,'Intercept',1)
X.insert(0,'Pair',train.Pair)

In [161]:
X['Pair'] = X['Pair'].astype(str)
test['Pair'] = test['Pair'].astype(str)
test = test.merge(X, how='left', on='Pair')

/Users/abigail/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [162]:
X_test = test[["Intercept", "NCA", "CN", "AA", "RA", "JC", "PA", "KI", "PR_s1", "PR_s2"]]
X_test

,Intercept,NCA,CN,AA,RA,JC,PA,KI,PR_s1,PR_s2
0,1.0,0.0,0.0,0.000000,0.000000,0.000000,56.0,-0.000552,0.000220,0.000185
1,1.0,0.0,0.0,0.000000,0.000000,0.000000,24.0,-0.000088,0.000220,0.000126
2,1.0,0.0,0.0,0.000000,0.000000,0.000000,496.0,0.009896,0.000291,0.000891
3,1.0,1.0,2.0,1.242670,0.400000,0.080000,72.0,0.057307,0.000549,0.000115
4,1.0,0.0,0.0,0.000000,0.000000,0.000000,391.0,-0.138788,0.000362,0.000546
...,...,...,...,...,...,...,...,...,...,...
1995,1.0,0.0,0.0,0.000000,0.000000,0.000000,2.0,-0.000290,0.000071,0.000084
1996,1.0,0.0,0.0,0.000000,0.000000,0.000000,2.0,0.000000,0.000085,0.000265
1997,1.0,0.0,0.0,0.000000,0.000000,0.000000,116.0,0.011228,0.000111,0.000745
1998,1.0,4.0,4.0,1.766048,0.424368,0.053333,370.0,0.049551,0.001486,0.000146


In [163]:
from scipy.special import expit #the logistic function
sigmoid = expit
Prob = sigmoid(X_test @ w_sklearn)
test["Predict_Prob"] = Prob 

In [164]:
Prediction = test[["Id", "Pair", "Predict_Prob"]]
Prediction

,Id,Pair,Predict_Prob
0,1,"(0, 2917)",4.292238e-33
1,2,"(0, 2956)",3.834547e-33
2,3,"(1, 4038)",2.102764e-32
3,4,"(2, 1848)",1.000000e+00
4,5,"(3, 513)",1.029004e-32
...,...,...,...
1995,1996,"(3865, 3924)",3.544278e-33
1996,1997,"(3917, 4025)",3.546635e-33
1997,1998,"(3922, 3947)",5.458936e-33
1998,1999,"(3955, 3987)",1.000000e+00
